In [ ]:
!pip install transformers
!pip install torch
!pip install beautifulsoup4
!pip install requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import T5Tokenizer, PegasusForConditionalGeneration, pipeline, AutoModelForQuestionAnswering, AutoTokenizer

In [ ]:
url = input("Enter the URL for the patent or paper you want to analyze:")
headers = {"User-Agent" : "Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.find("span", {"itemprop" : "title"}).text.strip()
abstract = soup.find("div", {"class" : "abstract"}).text.strip()
claims = soup.find_all("div", {"class" : "claim"})
claims_text = [claim.text.strip() for claim in claims]
claims_text = "\n".join(claims_text)

Enter the URL for the patent or paper you want to analyze:https://patents.google.com/patent/US9554790B2/en?q=(surgical+retractor+arm)&oq=surgical+retractor+arm


In [ ]:
# model_name = "google/pegasus-xsum"
# model_name = "t5-base"
# tokenizer = T5Tokenizer.from_pretrained(model_name)
# model = PegasusForConditionalGeneration.from_pretrained(model_name)
# summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)
model_name = "facebook/bart-large-cnn"
summarizer = pipeline("summarization", model = model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
print("abstract is: ", abstract)

abstract is:  Minimaily invasive surgical retractors and methods of using the retractors are provided. This retractor may be introduced through a sealed port, controlled by a robotic system, have full articulation, and need not require assembly within the patient's body. As a result, fully endoscopic mitral valve surgery may be performed.


In [ ]:
print("claims are: ", claims_text)

claims are:  1. A method of retracting tissue, the method performed by a processing system, the method comprising
receiving instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly, the instructions including
moving the retractor assembly to a closed configuration in which first and second blades of the retractor assembly have a first angular displacement about is pivot assembly and in which an expansion structure supported by the first and second blades is collapsed and
moving the retractor assembly to an open configuration in which first and second blades have a second angular displacement about the pivot assembly, wherein the second angular displacement is greater than the first angular displacement and in which the expansion structure is expanded to bound an expansion region, wherein the expansion region bounded by the expanded expansion structure is greater than the angular displacement regi

In [ ]:
summary = summarizer(abstract, max_length=max(40, len(abstract.split())//2), min_length=max(10, len(abstract.split())//5), do_sample=False, num_beams=4)[0]['summary_text']
print("summary is: ", summary)
abs_summary = summary

summary is:  This retractor may be introduced through a sealed port, controlled by a robotic system, and have full articulation. As a result, fully endoscopic mitral valve surgery may be performed


In [ ]:
# claims_text = summary + "\n" + claims_text
claims_text_arr = claims_text.split("\n")
claims_arr = []
count = 0
claims_str = ''
while count < len(claims_text_arr):
  while(len(claims_str.split()) < 20):
    claims_str += claims_text_arr[count] + " "
    count += 1
  claims_arr.append(claims_str)
  claims_str = ''
  count += 1
claims_text_arr = claims_arr
count = 0
# step = len(claims_text_arr)//500
claims_string = ''
# for claims_info in range(len(claims_text_arr)//300 + 1):
for claims_info in claims_text_arr:
  # if (count + 300 < len(claims_text_arr)):
  #   claims_info_str = " ".join(claims_text_arr[count:(count + 300)])
  # else:
  #   claims_info_str = " ".join(claims_text_arr[count:])
  claims_info = "Summarize this info in very simple layman's terms: " + claims_info
  summary = summarizer(claims_info, max_new_tokens=100, max_length=len(claims_info.split())//2, min_length=max(10, len(claims_info.split())//10), do_sample=False, num_beams=4)[0]['summary_text']
  count += 1
  claims_string += summary + "\n"
print("claims string is: ", claims_string)

Both `max_new_tokens` (=100) and `max_length`(=25) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=38) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=17) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=100) and `max_length`(=38) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both

claims string is:  A method of retracting tissue, the method performed by a processing system. A minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly.
Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement about the pivot assembly. The expansion structure is expanded to bound an expansion region.
Instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly.
Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement about the pivot assembly. The expansion structure is expanded to bound an expansion region.
The method of claim 1 wherein the first blade has a distal end coupled to a first linkage and to a second linkage. Summarize this info in very simple layman's terms.
The method of claim 4 wherein the second linkage is pi

In [ ]:
simplifier = pipeline("text2text-generation", model="google/flan-t5-base")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


simplification is:  Instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly. Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement about the pivot assembly. The expansion structure is expanded to bound an expansion region. The method of claim 1 wherein the first blade has a distal end coupled to a first linkage and to a second linkage. The method of claim 4 wherein the second linkage is pivotally coupled to a fourth linkage, wherein the fourth linkage extends through the first blade. The method of claim 1 wherein the retractor assembly includes webbing extending between the first and second blades. The method of claim 5 and 6 wherein the fifth linkage is coupled between thefirst and fourth linkages.


In [ ]:
prompt = "Explain this in the simplest language possible, like for a child: " + claims_string
simplification = simplifier(prompt)[0]['generated_text']
print("simplification is: ", simplification)

simplification is:  Instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly. Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement about the pivot assembly. The expansion structure is expanded to bound an expansion region. Instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly. Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement about the pivot assembly. The expansion structure is expanded to bound an expansion region. Instructions for operating a minimally invasive surgical instrument that includes an elongate shaft coupled at a distal end to a retractor assembly. Moving the retractor assembly to an open configuration. First and second blades have a second angular displacement abou

In [ ]:
# model_name = "deepset/bert-base-cased-squad2"
# bert_model = AutoModelForQuestionAnswering.from_pretrained(model_name)
# bert_tokenizer = AutoTokenizer.from_pretrained(model_name)
# question_answerer = pipeline("question-answering", model=bert_model, tokenizer=bert_tokenizer)

NameError: name 'AutoModelForQuestionAnswering' is not defined

In [ ]:
# info = abs_summary + "\n" + claims_string

In [ ]:
# question = input("What question do you want to ask")
# answer = question_answerer(question=question, context=info)
# print(answer)